In [67]:
%run Mylib_import.ipynb

exit_flag = False
def exit_visualization(vis):
    global exit_flag
    exit_flag = True
    return False

def get_depth_scale(pipeline):
    return pipeline.get_active_profile().get_device().first_depth_sensor().get_depth_scale()

def fetch_frame_from_camera(config, pipeline, dtrunc):
    frames = pipeline.wait_for_frames()
    
    # Align the depth frame to the color frame
    align_to = rs.stream.color
    align = rs.align(align_to)
    aligned_frames = align.process(frames)
    
    depth_frame = aligned_frames.get_depth_frame()
    color_frame = aligned_frames.get_color_frame()

    depth_image_np = np.asanyarray(depth_frame.get_data())
    color_image_np = np.asanyarray(color_frame.get_data())
    depth_image = o3d.geometry.Image(depth_image_np)
    color_image = o3d.geometry.Image(color_image_np)
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color_image, depth_image, depth_scale=1000.0, depth_trunc=dtrunc, convert_rgb_to_intensity=False)
    
    # Get intrinsic parameters
    profile = color_frame.get_profile()
    intrinsics = profile.as_video_stream_profile().get_intrinsics()
    intrinsic_o3d = o3d.camera.PinholeCameraIntrinsic(
        intrinsics.width, intrinsics.height,
        intrinsics.fx, intrinsics.fy,
        intrinsics.ppx, intrinsics.ppy)

    return rgbd_image, intrinsic_o3d, depth_image_np

def export_realtime_pointclouds_withvis():
    global exit_flag

    # Visualization setup
    vis = o3d.visualization.VisualizerWithKeyCallback()
    vis.create_window(width=800, height=600)
    vis.register_key_callback(256, exit_visualization)
    
    # Configurations for both cameras
    config_L515 = rs.config()
    config_L515.enable_device('f0352346') 
    config_L515.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
    config_L515.enable_stream(rs.stream.color, 640, 480, rs.format.rgb8, 30)
    
    config_D435i = rs.config()
    config_D435i.enable_device('841512070605')
    config_D435i.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
    config_D435i.enable_stream(rs.stream.color, 640, 480, rs.format.rgb8, 30)
    
    # Start pipelines for both cameras
    pipeline_L515 = rs.pipeline()
    pipeline_D435i = rs.pipeline()
    pipeline_L515.start(config_L515)
    pipeline_D435i.start(config_D435i)
    
    # Get scaling factors in Matrix
    depth_scale_L515 = get_depth_scale(pipeline_L515)
    depth_scale_D435i = get_depth_scale(pipeline_D435i)
    scaling_matrix_D435i = create_scaling_matrix(depth_scale_D435i/depth_scale_L515)
    
    frame_number = 0
    while not exit_flag:
        rgbd_L515, intr_L515, depth_np_L515 = fetch_frame_from_camera(
            config_L515, pipeline_L515, 4.0)
        rgbd_D435i, intr_D435i, depth_np_D435i = fetch_frame_from_camera(
            config_D435i, pipeline_D435i, 1.0)
        
        # Apply proper color mapping to depth frames
        depth_colormap_l515 = cv2.applyColorMap(cv2.convertScaleAbs(
            depth_np_L515, alpha=0.1), cv2.COLORMAP_JET)
        depth_colormap_d435i = cv2.applyColorMap(cv2.convertScaleAbs(
            depth_np_D435i, alpha=0.1), cv2.COLORMAP_JET)
        
        # Create pointcloud
        pcd_L515 = o3d.geometry.PointCloud.create_from_rgbd_image(
            rgbd_L515, intr_L515)
        pcd_D435i = o3d.geometry.PointCloud.create_from_rgbd_image(
            rgbd_D435i, intr_D435i)
        
        # Rotate/Scale Pointclouds
        pcd_L515.transform(flip_uptodown_matrix(0))
        pcd_D435i.transform(flip_uptodown_matrix(0))
        pcd_D435i.transform(scaling_matrix_D435i)
        
        # Combine the two Pointclouds
        combined_pcd = pcd_L515 + pcd_D435i
        
        # Update the Visualization
        vis.clear_geometries()
        vis.add_geometry(combined_pcd)
        vis.poll_events()
        vis.update_renderer()
        
        
        # Export point cloud
        pcd_filename_l515 = f'{dir_pcd}/l515/realdepth_{frame_number}.ply'
        rgb_filename_l515 = f'{dir_color}/l515/rgb_{frame_number}.png'
        depth_filename_l515 = f'{dir_depth}/l515/realdepth_{frame_number}.png'
        depthJET_filename_l515 = f'{dir_depth}/l515/depthcolormap_{frame_number}.png'
        o3d.io.write_point_cloud(pcd_filename_l515, pcd_L515)
        cv2.imwrite(rgb_filename_l515, cv2.cvtColor(np.asarray(rgbd_L515.color), cv2.COLOR_BGR2RGB))
        cv2.imwrite(depth_filename_l515, depth_np_L515.astype(np.uint16))
        cv2.imwrite(depthJET_filename_l515, depth_colormap_l515)
        
        pcd_filename_d435i = f'{dir_pcd}/d435i/realdepth_{frame_number}.ply'
        rgb_filename_d435i = f'{dir_color}/d435i/rgb_{frame_number}.png'
        depth_filename_d435i = f'{dir_depth}/d435i/realdepth_{frame_number}.png'
        depthJET_filename_d435i = f'{dir_depth}/d435i/depthcolormap_{frame_number}.png'
        o3d.io.write_point_cloud(pcd_filename_d435i, pcd_D435i)
        cv2.imwrite(rgb_filename_d435i, cv2.cvtColor(np.asarray(rgbd_D435i.color), cv2.COLOR_BGR2RGB))
        cv2.imwrite(depth_filename_d435i, depth_np_D435i.astype(np.uint16))
        cv2.imwrite(depthJET_filename_d435i, depth_colormap_d435i)
        frame_number += 1
        
    vis.destroy_window()
    pipeline_L515.stop()
    pipeline_D435i.stop()
    
    return pcd_L515, pcd_D435i, rgbd_L515, rgbd_D435i

# if __name__ == "__main__":
#     pcd_L515, pcd_D435i = export_realtime_pointclouds_withvis()

In [68]:
if __name__ == "__main__":
    #main save directories
    dir_main = "D:/Users/msari/realsense_export_frame"

    folname_new = input()
    #folname_new = 'test_bear01' #'testfol'
    create_folder(dir_main +'/'+ folname_new)

    #color, depth frames export
    dir_color = dir_main + '/'+ folname_new + '/color'
    dir_depth = dir_main + '/'+ folname_new + '/depth'
    dir_vis = dir_main + '/'+ folname_new + '/vis'
    dir_pcd = dir_main + '/'+ folname_new + '/pcd'
    create_folder(dir_color)
    create_folder(dir_depth)
    create_folder(dir_vis)
    create_folder(dir_pcd)

    #create results folder
    create_folder(dir_color+'/l515')
    create_folder(dir_depth+'/l515')
    create_folder(dir_pcd+'/l515')
    create_folder(dir_color+'/d435i')
    create_folder(dir_depth+'/d435i')
    create_folder(dir_pcd+'/d435i')
    
    exit_flag = False
    pcd_L515, pcd_D435i, rgbd_L515, rgbd_D435i = export_realtime_pointclouds_withvis()

Folder 'D:/Users/msari/realsense_export_frame/test_bear01' already exists.
Folder 'D:/Users/msari/realsense_export_frame/test_bear01/color' already exists.
Folder 'D:/Users/msari/realsense_export_frame/test_bear01/depth' already exists.
Folder 'D:/Users/msari/realsense_export_frame/test_bear01/vis' already exists.
Folder 'D:/Users/msari/realsense_export_frame/test_bear01/pcd' already exists.
Folder 'D:/Users/msari/realsense_export_frame/test_bear01/color/l515' already exists.
Folder 'D:/Users/msari/realsense_export_frame/test_bear01/depth/l515' already exists.
Folder 'D:/Users/msari/realsense_export_frame/test_bear01/pcd/l515' already exists.
Folder 'D:/Users/msari/realsense_export_frame/test_bear01/color/d435i' already exists.
Folder 'D:/Users/msari/realsense_export_frame/test_bear01/depth/d435i' already exists.
Folder 'D:/Users/msari/realsense_export_frame/test_bear01/pcd/d435i' already exists.


In [60]:
combine_pcd = pcd_L515 + pcd_D435i
o3d.visualization.draw_geometries([combine_pcd],width=800, height=600)